# DFT & eDMFT Calculations

## 1. Dependencies and imports

In [1]:
import importlib, shutil, os
import pandas as pd
from IPython.display import display

check_data = []

for pkg in ["numpy", "ase", "jarvis", "triqs", "nglview", "seaborn", "pandas"]:
    try:
        mod = importlib.import_module(pkg)
        version = getattr(mod, '__version__', 'N/A')
        check_data.append({'Category': 'Package', 'Item': pkg, 'Status': '✓', 'Details': version})
    except ImportError:
        check_data.append({'Category': 'Package', 'Item': pkg, 'Status': '✗', 'Details': 'Not installed'})

try:
    from triqs.gf import GfImFreq, inverse, iOmega_n, SemiCircular
    from triqs_cthyb import Solver
    from triqs.operators import n
    
    beta_test = 10.0
    U_test = 2.0
    S_test = Solver(beta=beta_test, gf_struct=[('up', 1), ('down', 1)])
    S_test.G_iw << SemiCircular(1.0)
    H_test = U_test * n('up', 0) * n('down', 0)
    
    for name, g0 in S_test.G0_iw:
        g0 << inverse(iOmega_n + 0.5 - 0.25 * S_test.G_iw[name])
    
    S_test.solve(h_int=H_test, n_cycles=100, length_cycle=10, n_warmup_cycles=50)
    density = S_test.G_iw['up'].density()[0, 0].real
    
    check_data.append({'Category': 'Package', 'Item': 'TRIQS CTHYB Runtime', 'Status': '✓', 'Details': f'Working (test density={density:.3f})'})
except Exception as e:
    check_data.append({'Category': 'Package', 'Item': 'TRIQS CTHYB Runtime', 'Status': '✗', 'Details': str(e)[:50]})

for exe in ["pw.x", "wannier90.x", "pw2wannier90.x", "mpirun"]:
    found = shutil.which(exe)
    if found:
        check_data.append({'Category': 'Executable', 'Item': exe, 'Status': '✓', 'Details': found})
    else:
        check_data.append({'Category': 'Executable', 'Item': exe, 'Status': '✗', 'Details': 'Not in PATH'})

pseudo_dir = "/usr/share/espresso/pseudo"
if os.path.isdir(pseudo_dir):
    num_pseudos = len([f for f in os.listdir(pseudo_dir) if f.endswith(('.upf', '.UPF'))])
    check_data.append({'Category': 'Pseudopotentials', 'Item': 'Directory', 'Status': '✓', 'Details': pseudo_dir})
    check_data.append({'Category': 'Pseudopotentials', 'Item': 'File count', 'Status': '✓', 'Details': f'{num_pseudos} files'})
else:
    check_data.append({'Category': 'Pseudopotentials', 'Item': 'Directory', 'Status': '✗', 'Details': 'Not found'})

df = pd.DataFrame(check_data)
display(df)


╔╦╗╦═╗╦╔═╗ ╔═╗  ┌─┐┌┬┐┬ ┬┬ ┬┌┐ 
 ║ ╠╦╝║║═╬╗╚═╗  │   │ ├─┤└┬┘├┴┐
 ╩ ╩╚═╩╚═╝╚╚═╝  └─┘ ┴ ┴ ┴ ┴ └─┘

The local Hamiltonian of the problem:
-0.5*c_dag('down',0)*c('down',0) + -0.5*c_dag('up',0)*c('up',0) + 2*c_dag('down',0)*c_dag('up',0)*c('up',0)*c('down',0)
Using autopartition algorithm to partition the local Hilbert space
Found 4 subspaces.

Warming up ...
03:00:58 100% ETA 00:00:00 cycle 49 of 50



Accumulating ...
03:00:58 100% ETA 00:00:00 cycle 99 of 100


[Rank 0] Collect results: Waiting for all mpi-threads to finish accumulating...
[Rank 0] Timings for all measures:
Measure               | seconds   
Auto-correlation time | 1.0466e-05
Average order         | 1.789e-06 
Average sign          | 1.829e-06 
G_tau measure         | 0.000297373
Total measure time    | 0.000311457
[Rank 0] Acceptance rate for all moves:
Move set Insert two operators: 0.13617
  Move  Insert Delta_up: 0.132812
  Move  Insert Delta_down: 0.140187
Move set Remove two operators: 0.188889
  Move  Remove Delt

Starting serial run at: 2025-10-16 03:00:58.968059


,Category,Item,Status,Details
0,Package,numpy,✓,2.0.2
1,Package,ase,✓,3.26.0
2,Package,jarvis,✓,2025.5.30
3,Package,triqs,✓,N/A
4,Package,nglview,✓,4.0
5,Package,seaborn,✓,0.13.2
6,Package,pandas,✓,2.3.3
7,Package,TRIQS CTHYB Runtime,✓,Working (test density=1.100)
8,Executable,pw.x,✓,/home/vm/miniconda3/envs/DSI/bin/pw.x
9,Executable,wannier90.x,✓,/home/vm/miniconda3/envs/DSI/bin/wannier90.x


In [2]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML, Markdown
from jarvis.db.figshare import data as jarvis_data
from jarvis.core.atoms import Atoms as JarvisAtoms
from ase import Atoms as AseAtoms
from triqs.gf import GfImFreq, BlockGf
import nglview as nv
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.patches import FancyBboxPatch
import seaborn as sns
import re
import sys
import subprocess
import tempfile
import random


sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['legend.fontsize'] = 9
plt.rcParams['figure.titlesize'] = 13

## 2. Config

In [3]:
# database
JARVIS_DATABASE = "dft_3d"

# working directory
WORK_DIR = os.path.join(os.getcwd(), "calculations")
os.makedirs(WORK_DIR, exist_ok=True)

# pseudopotentials directory
QE_PSEUDOPOTENTIALS_DIR = pseudo_dir

# executables (resolved from PATH)
QE_EXECUTABLE = shutil.which("pw.x") or shutil.which("pw")
WANNIER90_EXECUTABLE = shutil.which("wannier90.x") or shutil.which("wannier90")
PW2WANNIER90_EXECUTABLE = shutil.which("pw2wannier90.x") or shutil.which("pw2wannier90")
MPI_EXECUTABLE = shutil.which("mpirun") or shutil.which("mpiexec")

# params
QE_NPROCS = 8
QE_TIMEOUT = 600

# DFT params
ECUTWFC = 30.0
ECUTRHO = 120.0
K_POINTS = [2, 2, 2]
OCCUPATIONS = "smearing"
SMEARING = "gaussian"
DEGAUSS = 0.02

# wanneir params
NUM_WANNIER_FUNCTIONS = 10

# eDMFT params
HUBBARD_U = 4.0
TRIQS_BETA = 40.0
DMFT_ITERATIONS = 20

# energy thresholds
DISPLACEMENT_THRESHOLD_EV = 25.0
PKA_ENERGY_THRESHOLD_EV = 100.0

# Display configuration as a styled table
config_data = [
    {'Category': 'Paths', 'Parameter': 'WORK_DIR', 'Value': WORK_DIR},
    {'Category': 'Paths', 'Parameter': 'QE_PSEUDOPOTENTIALS_DIR', 'Value': QE_PSEUDOPOTENTIALS_DIR},
    {'Category': 'Database', 'Parameter': 'JARVIS_DATABASE', 'Value': JARVIS_DATABASE},
    {'Category': 'Executables', 'Parameter': 'QE_EXECUTABLE', 'Value': QE_EXECUTABLE},
    {'Category': 'Executables', 'Parameter': 'MPI_EXECUTABLE', 'Value': MPI_EXECUTABLE},
    {'Category': 'Executables', 'Parameter': 'WANNIER90_EXECUTABLE', 'Value': WANNIER90_EXECUTABLE},
    {'Category': 'Executables', 'Parameter': 'PW2WANNIER90_EXECUTABLE', 'Value': PW2WANNIER90_EXECUTABLE},
    {'Category': 'Runtime', 'Parameter': 'QE_NPROCS', 'Value': QE_NPROCS},
    {'Category': 'Runtime', 'Parameter': 'QE_TIMEOUT', 'Value': f'{QE_TIMEOUT} sec'},
    {'Category': 'DFT', 'Parameter': 'ECUTWFC', 'Value': f'{ECUTWFC} eV'},
    {'Category': 'DFT', 'Parameter': 'ECUTRHO', 'Value': f'{ECUTRHO} eV'},
    {'Category': 'DFT', 'Parameter': 'K_POINTS', 'Value': f'{K_POINTS}'},
    {'Category': 'DFT', 'Parameter': 'OCCUPATIONS', 'Value': OCCUPATIONS},
    {'Category': 'DFT', 'Parameter': 'SMEARING', 'Value': SMEARING},
    {'Category': 'DFT', 'Parameter': 'DEGAUSS', 'Value': DEGAUSS},
    {'Category': 'Wannier90', 'Parameter': 'NUM_WANNIER_FUNCTIONS', 'Value': NUM_WANNIER_FUNCTIONS},
    {'Category': 'eDMFT', 'Parameter': 'HUBBARD_U', 'Value': f'{HUBBARD_U} eV'},
    {'Category': 'eDMFT', 'Parameter': 'TRIQS_BETA', 'Value': f'{TRIQS_BETA} eV⁻¹'},
    {'Category': 'eDMFT', 'Parameter': 'DMFT_ITERATIONS', 'Value': DMFT_ITERATIONS},
    {'Category': 'Thresholds', 'Parameter': 'DISPLACEMENT_THRESHOLD_EV', 'Value': f'{DISPLACEMENT_THRESHOLD_EV} eV'},
    {'Category': 'Thresholds', 'Parameter': 'PKA_ENERGY_THRESHOLD_EV', 'Value': f'{PKA_ENERGY_THRESHOLD_EV} eV'},
]

df_config = pd.DataFrame(config_data)
display(HTML("<h3>Configuration Summary</h3>"))
display(df_config.style.set_properties(**{'text-align': 'left'}).set_table_styles([
    {'selector': 'th', 'props': [('color', 'white'), ('font-weight', 'bold')]}
]).hide(axis='index'))

Category,Parameter,Value
Paths,WORK_DIR,/home/vm/DSI_Project3/calculations
Paths,QE_PSEUDOPOTENTIALS_DIR,/usr/share/espresso/pseudo
Database,JARVIS_DATABASE,dft_3d
Executables,QE_EXECUTABLE,/home/vm/miniconda3/envs/DSI/bin/pw.x
Executables,MPI_EXECUTABLE,/usr/bin/mpirun
Executables,WANNIER90_EXECUTABLE,/home/vm/miniconda3/envs/DSI/bin/wannier90.x
Executables,PW2WANNIER90_EXECUTABLE,/home/vm/miniconda3/envs/DSI/bin/pw2wannier90.x
Runtime,QE_NPROCS,8
Runtime,QE_TIMEOUT,600 sec
DFT,ECUTWFC,30.0 eV


## 3. Load Material from JARVIS

In [4]:
data = jarvis_data(JARVIS_DATABASE)

# For testing: Use specific material instead of random selection
# mat = random.choice(data)
MATERIAL_ID = 'JVASP-25075'  # Testing with specific material
mat = next((item for item in data if item.get('jid') == MATERIAL_ID), None)
if mat is None:
    raise ValueError(f"Material {MATERIAL_ID} not found in {JARVIS_DATABASE} database")

jarvis_atoms = JarvisAtoms.from_dict(mat['atoms'])
ase_atoms = AseAtoms(
    symbols=jarvis_atoms.elements,
    positions=jarvis_atoms.cart_coords,
    cell=jarvis_atoms.lattice_mat,
    pbc=True
)

Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699
Loading the zipfile...
Loading completed.
Loading completed.


In [5]:
# Extract detailed material information
elements = list(set(ase_atoms.get_chemical_symbols()))
element_counts = {e: ase_atoms.get_chemical_symbols().count(e) for e in elements}
cell_volume = ase_atoms.get_volume()
density = sum(ase_atoms.get_masses()) / cell_volume
cell_lengths = ase_atoms.cell.lengths()
cell_angles = ase_atoms.cell.angles()

# Create material properties table
mat_props = [
    {'Property': 'Material ID', 'Value': MATERIAL_ID},
    {'Property': 'Chemical Formula', 'Value': ase_atoms.get_chemical_formula()},
    {'Property': 'Total Atoms', 'Value': len(ase_atoms)},
    {'Property': 'Unique Elements', 'Value': ', '.join(sorted(elements))},
    {'Property': 'Element Composition', 'Value': str(element_counts)},
    {'Property': 'Cell Volume', 'Value': f'{cell_volume:.3f} Å³'},
    {'Property': 'Density', 'Value': f'{density:.3f} amu/Å³'},
    {'Property': 'Cell Lengths (a, b, c)', 'Value': f'{cell_lengths[0]:.3f}, {cell_lengths[1]:.3f}, {cell_lengths[2]:.3f} Å'},
    {'Property': 'Cell Angles (α, β, γ)', 'Value': f'{cell_angles[0]:.2f}°, {cell_angles[1]:.2f}°, {cell_angles[2]:.2f}°'},
]

# Add JARVIS metadata if available
if 'formation_energy_peratom' in mat:
    mat_props.append({'Property': 'Formation Energy', 'Value': f'{mat["formation_energy_peratom"]:.4f} eV/atom'})
if 'optb88vdw_bandgap' in mat:
    mat_props.append({'Property': 'Band Gap (OptB88vdW)', 'Value': f'{mat["optb88vdw_bandgap"]:.4f} eV'})
if 'spillage' in mat:
    mat_props.append({'Property': 'Spillage', 'Value': f'{mat["spillage"]:.4f}'})
if 'kpoint_length_unit' in mat:
    mat_props.append({'Property': 'K-point Length Unit', 'Value': f'{mat["kpoint_length_unit"]}'})

df_material = pd.DataFrame(mat_props)
display(HTML(f"<h3>Material Properties: {MATERIAL_ID}</h3>"))
display(df_material.style.set_properties(**{'text-align': 'left'}).set_table_styles([
    {'selector': 'th', 'props': [('color', 'white'), ('font-weight', 'bold')]}
]).hide(axis='index'))

Property,Value
Material ID,JVASP-25075
Chemical Formula,Te2
Total Atoms,2
Unique Elements,Te
Element Composition,{'Te': 2}
Cell Volume,66.084 Å³
Density,3.862 amu/Å³
"Cell Lengths (a, b, c)","4.538, 4.538, 4.538 Å"
"Cell Angles (α, β, γ)","60.00°, 60.00°, 60.00°"
Formation Energy,0.0383 eV/atom


In [6]:
material_data = {
    'material_id': MATERIAL_ID,
    'symbols': ase_atoms.get_chemical_symbols(),
    'positions': ase_atoms.positions.tolist(),
    'cell': ase_atoms.cell.tolist(),
    'formula': ase_atoms.get_chemical_formula(),
}

In [7]:
from ase.build import make_supercell
supercell = make_supercell(ase_atoms, [[2, 0, 0], [0, 2, 0], [0, 0, 2]])
view = nv.show_ase(supercell)
view.add_unitcell()
view.center()
display(view)

NGLWidget()

## 4. Generate Quantum ESPRESSO Input Files (SCF & NSCF)

In [8]:
elements = list(set(ase_atoms.get_chemical_symbols()))
pseudos = {}
for e in elements:
    candidates = [f for f in os.listdir(QE_PSEUDOPOTENTIALS_DIR) 
                  if f.startswith(e+'.') or f.startswith(e+'_') or 
                     f.startswith(e.lower()+'.') or f.startswith(e.lower()+'_')]
    pseudos[e] = candidates[0]

print(f"Material: {MATERIAL_ID}")
print(f"Elements: {elements}")
print(f"Pseudopotentials: {list(pseudos.values())}")

# generate k-points for nscf (explicit, wannier90-compatible)
nk_nscf = [K_POINTS[0]*2, K_POINTS[1]*2, K_POINTS[2]*2]
nkpts_nscf = nk_nscf[0] * nk_nscf[1] * nk_nscf[2]
kpoints_nscf = []
for k in range(nk_nscf[2]):
    for j in range(nk_nscf[1]):
        for i in range(nk_nscf[0]):
            kpoints_nscf.append([i/nk_nscf[0], j/nk_nscf[1], k/nk_nscf[2]])

def generate_qe_input(calc_type, k_grid, atoms, material_id, pseudos, nbnd=None):
    lines = []
    lines.append(f"&CONTROL\n  calculation='{calc_type}'\n  prefix='{material_id}'\n")
    lines.append(f"  outdir='./tmp'\n  pseudo_dir='{QE_PSEUDOPOTENTIALS_DIR}'\n/\n")
    lines.append(f"&SYSTEM\n  ibrav=0\n  nat={len(atoms)}\n  ntyp={len(elements)}\n")
    lines.append(f"  ecutwfc={ECUTWFC}\n  ecutrho={ECUTRHO}\n")
    lines.append(f"  occupations='{OCCUPATIONS}'\n  smearing='{SMEARING}'\n  degauss={DEGAUSS}\n")
    if calc_type == 'nscf':
        lines.append("  nosym=.true.\n  noinv=.true.\n")
        # Set nbnd if provided, otherwise let QE auto-calculate
        if nbnd is not None:
            lines.append(f"  nbnd={nbnd}\n")
    lines.append("/\n&ELECTRONS\n  conv_thr=1.0d-6\n/\nATOMIC_SPECIES\n")
    for e in elements:
        mass = atoms.get_masses()[atoms.get_chemical_symbols().index(e)]
        lines.append(f"  {e}  {mass:.4f}  {pseudos[e]}\n")
    lines.append("\nCELL_PARAMETERS angstrom\n")
    for i in range(3):
        lines.append(f"  {atoms.cell[i,0]:16.10f} {atoms.cell[i,1]:16.10f} {atoms.cell[i,2]:16.10f}\n")
    lines.append("\nATOMIC_POSITIONS angstrom\n")
    for s, p in zip(atoms.get_chemical_symbols(), atoms.positions):
        lines.append(f"  {s:4s} {p[0]:16.10f} {p[1]:16.10f} {p[2]:16.10f}\n")
    if calc_type == 'scf':
        lines.append(f"\nK_POINTS automatic\n  {k_grid[0]} {k_grid[1]} {k_grid[2]}  0 0 0\n")
    else:
        lines.append(f"\nK_POINTS crystal\n  {nkpts_nscf}\n")
        for kpt in kpoints_nscf:
            lines.append(f"  {kpt[0]:16.10f} {kpt[1]:16.10f} {kpt[2]:16.10f}  1.0\n")
    return ''.join(lines)

# Generate SCF input
qe_scf_input = generate_qe_input('scf', K_POINTS, ase_atoms, MATERIAL_ID, pseudos)

# Generate initial NSCF input (will regenerate after SCF with correct nbnd)
qe_nscf_input = generate_qe_input('nscf', nk_nscf, ase_atoms, MATERIAL_ID, pseudos)

Material: JVASP-25075
Elements: ['Te']
Pseudopotentials: ['Te_pbe_v1.uspp.F.UPF']


## 5. Run DFT Calculations

In [9]:
def run_streaming(cmd, cwd, timeout=None):
    process = subprocess.Popen(
        cmd,
        shell=True,
        cwd=cwd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )
    output_lines = []
    for line in process.stdout:
        print(line, end='', flush=True)
        output_lines.append(line)
    
    process.wait(timeout=timeout)
    return ''.join(output_lines), process.returncode

# create material-specific directory in persistent location
dft_tmp_dir = os.path.join(WORK_DIR, f"dft_{MATERIAL_ID}")
os.makedirs(dft_tmp_dir, exist_ok=True)
print(f"Running DFT in: {dft_tmp_dir}\n")

scf_input_path = os.path.join(dft_tmp_dir, f"{MATERIAL_ID}_scf.in")
with open(scf_input_path, 'w') as f:
    f.write(qe_scf_input)

scf_output, _ = run_streaming(
    f"{MPI_EXECUTABLE} -np {QE_NPROCS} {QE_EXECUTABLE} -in {MATERIAL_ID}_scf.in",
    cwd=dft_tmp_dir,
    timeout=QE_TIMEOUT
)

# Extract number of bands from SCF output to use for NSCF
nbnd_match = re.search(r'number of Kohn-Sham states\s*=\s*(\d+)', scf_output)
if nbnd_match:
    scf_nbnd = int(nbnd_match.group(1))
    # Use enough bands for Wannier90 but ensure it's reasonable
    nscf_nbnd = max(scf_nbnd, NUM_WANNIER_FUNCTIONS * 3)
    print(f"\nSCF used {scf_nbnd} bands")
    print(f"Setting NSCF to {nscf_nbnd} bands (max of SCF and {NUM_WANNIER_FUNCTIONS * 3})\n")
else:
    # Fallback: use a safe number
    nscf_nbnd = NUM_WANNIER_FUNCTIONS * 4
    print(f"\nCouldn't extract nbnd from SCF, using conservative value: {nscf_nbnd}\n")

# Regenerate NSCF input with correct nbnd
qe_nscf_input = generate_qe_input('nscf', nk_nscf, ase_atoms, MATERIAL_ID, pseudos, nbnd=nscf_nbnd)

nscf_input_path = os.path.join(dft_tmp_dir, f"{MATERIAL_ID}_nscf.in")
with open(nscf_input_path, 'w') as f:
    f.write(qe_nscf_input)

nscf_output, _ = run_streaming(
    f"{MPI_EXECUTABLE} -np {QE_NPROCS} {QE_EXECUTABLE} -in {MATERIAL_ID}_nscf.in",
    cwd=dft_tmp_dir,
    timeout=QE_TIMEOUT
)

Running DFT in: /home/vm/DSI_Project3/calculations/dft_JVASP-25075

[vm-QEMU-Virtual-Machine][[4353,1],2][../../../../../../opal/mca/btl/tcp/btl_tcp_proc.c:266:mca_btl_tcp_proc_create_interface_graph] Unable to find reachable pairing between local and remote interfaces
[vm-QEMU-Virtual-Machine][[4353,1],1][../../../../../../opal/mca/btl/tcp/btl_tcp_proc.c:266:mca_btl_tcp_proc_create_interface_graph] Unable to find reachable pairing between local and remote interfaces
[vm-QEMU-Virtual-Machine][[4353,1],1][../../../../../../opal/mca/btl/tcp/btl_tcp_proc.c:266:mca_btl_tcp_proc_create_interface_graph] Unable to find reachable pairing between local and remote interfaces
[vm-QEMU-Virtual-Machine][[4353,1],2][../../../../../../opal/mca/btl/tcp/btl_tcp_proc.c:266:mca_btl_tcp_proc_create_interface_graph] Unable to find reachable pairing between local and remote interfaces
[vm-QEMU-Virtual-Machine][[4353,1],1][../../../../../../opal/mca/btl/tcp/btl_tcp_proc.c:266:mca_btl_tcp_proc_create_interfac

In [10]:
dft_results = {
    'scf_output': scf_output[:5000] + '...(truncated)' if len(scf_output) > 5000 else scf_output,
    'nscf_output': nscf_output[:5000] + '...(truncated)' if len(nscf_output) > 5000 else nscf_output,
    'tmp_dir': dft_tmp_dir
}

## 6. Generate Wannier90 Input

In [11]:
seedname = "wan"
nk = [K_POINTS[0]*2, K_POINTS[1]*2, K_POINTS[2]*2]
nkpts = nk[0] * nk[1] * nk[2]
kpoints = []
for k in range(nk[2]):
    for j in range(nk[1]):
        for i in range(nk[0]):
            kpoints.append([i/nk[0], j/nk[1], k/nk[2]])

wannier_input_lines = []
wannier_input_lines.append(f"num_wann = {NUM_WANNIER_FUNCTIONS}\n")
wannier_input_lines.append(f"num_bands = {nscf_nbnd}\n")  # Use actual nbnd from NSCF
wannier_input_lines.append("num_iter = 100\n")
wannier_input_lines.append("write_hr = true\n")
print(f"Wannier90 will use num_wann={NUM_WANNIER_FUNCTIONS}, num_bands={nscf_nbnd}")
wannier_input_lines.append("begin projections\nrandom\nend projections\n")
wannier_input_lines.append("begin unit_cell_cart\nang\n")
for i in range(3):
    wannier_input_lines.append(f"{ase_atoms.cell[i,0]:16.10f} {ase_atoms.cell[i,1]:16.10f} {ase_atoms.cell[i,2]:16.10f}\n")
wannier_input_lines.append("end unit_cell_cart\n")
wannier_input_lines.append("begin atoms_frac\n")
for s, p in zip(ase_atoms.get_chemical_symbols(), ase_atoms.get_scaled_positions()):
    wannier_input_lines.append(f"{s} {p[0]:.10f} {p[1]:.10f} {p[2]:.10f}\n")
wannier_input_lines.append("end atoms_frac\n")
wannier_input_lines.append(f"mp_grid = {nk[0]} {nk[1]} {nk[2]}\n")
wannier_input_lines.append("begin kpoints\n")
for kpt in kpoints:
    wannier_input_lines.append(f"{kpt[0]:16.10f} {kpt[1]:16.10f} {kpt[2]:16.10f}\n")
wannier_input_lines.append("end kpoints\n")

wannier_input = ''.join(wannier_input_lines)

Wannier90 will use num_wann=10, num_bands=30


## 7. Run Wannier90 Workflow

In [12]:
# create wannier-specific directory in persistent location
wan_tmpdir = os.path.join(WORK_DIR, f"wannier_{MATERIAL_ID}")
os.makedirs(wan_tmpdir, exist_ok=True)
print(f"Running Wannier90 in: {wan_tmpdir}\n")

win_file = os.path.join(wan_tmpdir, f"{seedname}.win")
with open(win_file, 'w') as f:
    f.write(wannier_input)

pp_output, _ = run_streaming(f"{WANNIER90_EXECUTABLE} -pp {seedname}", cwd=wan_tmpdir)

pw2wan_input = f"""&inputpp
  outdir = '{dft_tmp_dir}/tmp'
  prefix = '{MATERIAL_ID}'
  seedname = '{seedname}'
  write_mmn = .true.
  write_amn = .true.
  write_unk = .false.
/
"""
pw2wan_file = os.path.join(wan_tmpdir, "pw2wan.in")
with open(pw2wan_file, 'w') as f:
    f.write(pw2wan_input)

pw2wan_output, _ = run_streaming(f"{PW2WANNIER90_EXECUTABLE} < pw2wan.in", cwd=wan_tmpdir)

wannier_output, _ = run_streaming(f"{WANNIER90_EXECUTABLE} {seedname}", cwd=wan_tmpdir)

hr_file = os.path.join(wan_tmpdir, f"{seedname}_hr.dat")
if not os.path.exists(hr_file):
    print(f"\nError: {seedname}_hr.dat not found!")
    print(f"Contents of {wan_tmpdir}:")
    for item in os.listdir(wan_tmpdir):
        print(f"  {item}")
    raise FileNotFoundError(f"Wannier90 did not generate {seedname}_hr.dat")
    
with open(hr_file, 'r') as f:
    hr_data = f.read()

Running Wannier90 in: /home/vm/DSI_Project3/calculations/wannier_JVASP-25075


     Program PW2WANNIER v.7.3.1 starts on 16Oct2025 at  3: 1:10 

     This program is part of the open-source Quantum ESPRESSO suite
     for quantum simulation of materials; please cite
         "P. Giannozzi et al., J. Phys.:Condens. Matter 21 395502 (2009);
         "P. Giannozzi et al., J. Phys.:Condens. Matter 29 465901 (2017);
         "P. Giannozzi et al., J. Chem. Phys. 152 154105 (2020);
          URL http://www.quantum-espresso.org", 
     in publications or presentations arising from this work. More details at
     http://www.quantum-espresso.org/quote

     Parallel version (MPI & OpenMP), running on      10 processor cores
     Number of MPI processes:                 1
     Threads/MPI process:                    10

     MPI processes distributed on     1 nodes
     540 MiB available memory on the printing compute node when the environment starts


     Reading nscf_save data

     Reading xm

In [13]:
wannier_results = {
    'output': wannier_output[:5000] + '...(truncated)' if len(wannier_output) > 5000 else wannier_output,
    'hr_dat': hr_data
}

## 8. Parse Tight-Binding Hamiltonian

In [14]:
lines = hr_data.split('\n')
num_wann = int(lines[1].strip())
nrpts = int(lines[2].strip())
ndegen_lines = (nrpts + 14) // 15

h_local = np.zeros((num_wann, num_wann), dtype=complex)
for line in lines[3+ndegen_lines:]:
    parts = line.split()
    if len(parts) >= 7 and all(int(parts[i]) == 0 for i in range(3)):
        i, j = int(parts[3])-1, int(parts[4])-1
        h_local[i, j] = float(parts[5]) + 1j*float(parts[6])

h_local = 0.5 * (h_local + h_local.conj().T)
dft_energy = np.real(np.trace(h_local))

print(f"Parsed tight-binding Hamiltonian: {num_wann}×{num_wann}")
print(f"DFT energy (trace): {dft_energy:.6f} eV")

# Store for visualization
h_local_dft = h_local.copy()

Parsed tight-binding Hamiltonian: 10×10
DFT energy (trace): 73.737242 eV


## 9. FULL eDMFT with TRIQS CTHYB (Quantum Monte Carlo)

In [15]:
from triqs_cthyb import Solver
from triqs.operators import n
from triqs.gf import inverse, iOmega_n, SemiCircular
import time

num_wann_qmc = 1
h_local_qmc = h_local[:num_wann_qmc, :num_wann_qmc]
mu_qmc = np.real(np.trace(h_local_qmc)) / num_wann_qmc
t_eff = 1.0

print(f"DMFT: β={TRIQS_BETA}, U={HUBBARD_U}, μ={mu_qmc:.3f}, orbitals={num_wann_qmc}")

gf_struct = [('up', num_wann_qmc), ('down', num_wann_qmc)]
S = Solver(beta=TRIQS_BETA, gf_struct=gf_struct)
S.G_iw << SemiCircular(2*t_eff)

H_int = sum(HUBBARD_U * n('up', orb) * n('down', orb) for orb in range(num_wann_qmc))
qmc_params = {'h_int': H_int, 'n_cycles': 10000, 'length_cycle': 100, 'n_warmup_cycles': 5000}

print(f"Running {DMFT_ITERATIONS} iterations...\n")
start_time = time.time()

for iteration in range(DMFT_ITERATIONS):
    iter_start = time.time()
    
    g = 0.5 * (S.G_iw['up'] + S.G_iw['down'])
    for name, g0 in S.G0_iw:
        g0 << inverse(iOmega_n + mu_qmc - t_eff**2 * g)
    
    S.solve(**qmc_params)
    
    density_up = S.G_iw['up'].density()[0, 0].real
    density_down = S.G_iw['down'].density()[0, 0].real
    
    if (iteration + 1) % 5 == 0:
        print(f"Iter {iteration+1:2d}: n↑={density_up:.4f}, n↓={density_down:.4f}, t={time.time()-iter_start:.1f}s")

total_time = time.time() - start_time
print(f"\nCompleted in {total_time:.1f}s ({total_time/DMFT_ITERATIONS:.1f}s/iter)")
print(f"Final density: {density_up + density_down:.4f}")

DMFT: β=40.0, U=4.0, μ=5.265, orbitals=1
Running 20 iterations...


╔╦╗╦═╗╦╔═╗ ╔═╗  ┌─┐┌┬┐┬ ┬┬ ┬┌┐ 
 ║ ╠╦╝║║═╬╗╚═╗  │   │ ├─┤└┬┘├┴┐
 ╩ ╩╚═╩╚═╝╚╚═╝  └─┘ ┴ ┴ ┴ ┴ └─┘

The local Hamiltonian of the problem:
-5.26541*c_dag('down',0)*c('down',0) + -5.26541*c_dag('up',0)*c('up',0) + 4*c_dag('down',0)*c_dag('up',0)*c('up',0)*c('down',0)
Using autopartition algorithm to partition the local Hilbert space
Found 4 subspaces.

Warming up ...
03:01:56  27% ETA 00:00:00 cycle 1349 of 5000
03:01:56 100% ETA 00:00:00 cycle 4999 of 5000



Accumulating ...
03:01:56  12% ETA 00:00:00 cycle 1242 of 10000
03:01:57 100% ETA 00:00:00 cycle 9999 of 10000


[Rank 0] Collect results: Waiting for all mpi-threads to finish accumulating...
[Rank 0] Timings for all measures:
Measure               | seconds   
Auto-correlation time | 0.00122625
Average order         | 0.000348198
Average sign          | 0.000328127
G_tau measure         | 0.0131842 
Total measure time    | 0.0150868 
[Rank 0] Acceptance rate for all